In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)


In [ ]:
def load_data():
    data = pd.read_csv('combined_patna_big.csv')
    return data

In [ ]:
tweet_df = load_data()
tweet_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1.3933E+18,1.3933E+18,1.62102E+12,14/05/2021 20:25,0.0,NaN,@_A113N 😼 going for my second dose next Sunday,en,[],[],3162614958,3162614958,kyozetsv,ダニエル,5,20.0,https://twitter.com/kyozetsv/status/1393301539356221440,[],[],0,NaN,FALSE,1.0,1.0,0.0,NaN,second dose,NaN,"25.5941,85.1376,100mi",NaN,NaN,NaN,NaN,"[{'screen_name': '_A113N', 'name': 'allen\U0001f9a3📈', 'id': '1846779817'}]",NaN,NaN,NaN,NaN
1,1.3933E+18,1.3933E+18,1.62102E+12,14/05/2021 20:07,0.0,NaN,"After getting Second Dose of Vaccine, Sallu Bhai be like: ""Corona से डर नहीं लगता साहब, Radhe क...",und,"['radhe', 'radhereview']",[],2597232165,2597232165,RJPINKY935,RJ Pinky,5,20.0,https://twitter.com/RJPINKY935/status/1393296918399250432,[],[],0,NaN,FALSE,9.0,4.0,1.0,NaN,second dose,NaN,"25.5941,85.1376,100mi",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1.39328E+18,1.39326E+18,1.62102E+12,14/05/2021 18:49,0.0,NaN,@AgrawalRMD @IndiaToday @sardesairajdeep @mohanbhagwat @DrHVoffice @AmitShah @CDCgov @WHO @gavi ...,en,[],[],597935746,597935746,ARVINDTRIPATH17,ARVIND TRIPATHI,5,18.0,https://twitter.com/ARVINDTRIPATH17/status/1393277391926034432,[],[],0,NaN,FALSE,0.0,0.0,0.0,NaN,second dose,NaN,"25.5941,85.1376,100mi",NaN,NaN,NaN,NaN,"[{'screen_name': 'AgrawalRMD', 'name': 'Dr.radha mohan das agrawal', 'id': '1078766551'}, {'scre...",NaN,NaN,NaN,NaN
3,1.39327E+18,1.39327E+18,1.62102E+12,14/05/2021 18:08,0.0,NaN,@alok_ajay @chitraaum @ravishndtv @news24tvchannel Dr Ajay Alok ji mera first dose covishield k...,hi,[],[],620255405,620255405,vijaylibran6,Bhakt Indian,5,18.0,https://twitter.com/vijaylibran6/status/1393267008913907712,[],[],0,NaN,FALSE,0.0,0.0,0.0,NaN,second dose,NaN,"25.5941,85.1376,100mi",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1.39321E+18,1.39321E+18,1.621E+12,14/05/2021 14:24,0.0,NaN,Still haven't got second dose . Why this kind of issue ??? @ https://t.co/HgkWO6EZy4,en,[],[],951114582,951114582,AlokIndiann,Alok Kumar Gupta,5,14.0,https://twitter.com/AlokIndiann/status/1393210525182357507,[],['https://pbs.twimg.com/media/E1WtV7LUUAA9vLL.jpg'],1,https://pbs.twimg.com/media/E1WtV7LUUAA9vLL.jpg,FALSE,0.0,0.0,0.0,NaN,second dose,NaN,"25.5941,85.1376,100mi",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [ ]:
tweet_df = tweet_df.drop(['id','conversation_id','created_at','timezone','place','cashtags','user_id_str','username','name','day','hour','link','urls','photos','video','thumbnail','quote_url','search','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to','retweet_date','translate','trans_src','trans_dest'],axis = 1)

In [ ]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4587 entries, 0 to 4586
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       4587 non-null   object 
 1   tweet      4587 non-null   object 
 2   language   4587 non-null   object 
 3   hashtags   4587 non-null   object 
 4   user_id    4587 non-null   object 
 5   retweet    4587 non-null   object 
 6   nlikes     4587 non-null   float64
 7   nreplies   4587 non-null   float64
 8   nretweets  4587 non-null   float64
dtypes: float64(3), object(6)
memory usage: 322.6+ KB


In [ ]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)

Dataset size: (4591, 9)
Columns are: Index(['date', 'tweet', 'language', 'hashtags', 'user_id', 'retweet', 'nlikes',
       'nreplies', 'nretweets'],
      dtype='object')


In [ ]:
tweet_df.isnull().sum()

date         0
tweet        0
language     0
hashtags     0
user_id      0
retweet      0
nlikes       0
nreplies     0
nretweets    0
dtype: int64

In [ ]:
tweet_df.dropna(inplace=True)

In [ ]:
tweet_df.reset_index(drop=True,inplace=True)

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: remove_punct(x))
tweet_df.head(10)

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0
1,14/05/2021 20:07,After getting Second Dose of Vaccine Sallu Bhai be like Corona से डर नहीं लगता साहब Radhe के re...,und,"['radhe', 'radhereview']",2597232165,FALSE,9.0,4.0,1.0
2,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0
3,14/05/2021 18:08,alokajay chitraaum ravishndtv newstvchannel Dr Ajay Alok ji mera first dose covishield ka laga ...,hi,[],620255405,FALSE,0.0,0.0,0.0
4,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0
5,14/05/2021 12:46,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0
6,14/05/2021 10:49,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0
7,14/05/2021 10:42,Pfizer and BioNTech Confirm High Efficacy and No Serious Safety Concerns Through Up to Six Month...,en,[],1.29587E+18,FALSE,0.0,0.0,0.0
8,14/05/2021 10:36,Should I take the second dose Cause due to first dose many people fall ill SwasthyaMantra htt...,en,['swasthyamantra'],1.27788E+18,FALSE,0.0,0.0,0.0
9,14/05/2021 9:44,My husbands second dose is due but now centers in thane are saying days gap What to do Should ...,en,[],8.8721E+17,FALSE,25.0,12.0,3.0


In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

tweet_df['Tweet_tokenized'] = tweet_df['tweet'].apply(lambda x: tokenization(x.lower()))
tweet_df.head()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]"
1,14/05/2021 20:07,After getting Second Dose of Vaccine Sallu Bhai be like Corona से डर नहीं लगता साहब Radhe के re...,und,"['radhe', 'radhereview']",2597232165,FALSE,9.0,4.0,1.0,"[after, getting, second, dose, of, vaccine, sallu, bhai, be, like, corona, स, डर, नह, लगत, स, हब..."
2,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,..."
3,14/05/2021 18:08,alokajay chitraaum ravishndtv newstvchannel Dr Ajay Alok ji mera first dose covishield ka laga ...,hi,[],620255405,FALSE,0.0,0.0,0.0,"[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie..."
4,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]"


In [ ]:
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tweet_df['Tweet_nonstop'] = tweet_df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweet_df.head(10)

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]"
1,14/05/2021 20:07,After getting Second Dose of Vaccine Sallu Bhai be like Corona से डर नहीं लगता साहब Radhe के re...,und,"['radhe', 'radhereview']",2597232165,FALSE,9.0,4.0,1.0,"[after, getting, second, dose, of, vaccine, sallu, bhai, be, like, corona, स, डर, नह, लगत, स, हब...","[getting, second, dose, vaccine, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radhe, क, rev..."
2,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj..."
3,14/05/2021 18:08,alokajay chitraaum ravishndtv newstvchannel Dr Ajay Alok ji mera first dose covishield ka laga ...,hi,[],620255405,FALSE,0.0,0.0,0.0,"[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie..."
4,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]"
5,14/05/2021 12:46,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0,"[today, beingsalmankhan, took, his, second, dose, of, vaccine, and, i, took, my, first, dose, of...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc..."
6,14/05/2021 10:49,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0,"[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni..."
7,14/05/2021 10:42,Pfizer and BioNTech Confirm High Efficacy and No Serious Safety Concerns Through Up to Six Month...,en,[],1.29587E+18,FALSE,0.0,0.0,0.0,"[pfizer, and, biontech, confirm, high, efficacy, and, no, serious, safety, concerns, through, up...","[pfizer, biontech, confirm, high, efficacy, serious, safety, concerns, six, months, following, s..."
8,14/05/2021 10:36,Should I take the second dose Cause due to first dose many people fall ill SwasthyaMantra htt...,en,['swasthyamantra'],1.27788E+18,FALSE,0.0,0.0,0.0,"[should, i, take, the, second, dose, cause, due, to, first, dose, many, people, fall, ill, swast...","[take, second, dose, cause, due, first, dose, many, people, fall, ill, swasthyamantra, httpstcor..."
9,14/05/2021 9:44,My husbands second dose is due but now centers in thane are saying days gap What to do Should ...,en,[],8.8721E+17,FALSE,25.0,12.0,3.0,"[my, husbands, second, dose, is, due, but, now, centers, in, thane, are, saying, days, gap, what...","[husbands, second, dose, due, centers, thane, saying, days, gap, wait, keep, trying, drmanita, s..."


In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweet_df['Tweet_stemmed'] = tweet_df['Tweet_nonstop'].apply(lambda x: stemming(x))
tweet_df.head()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]"
1,14/05/2021 20:07,After getting Second Dose of Vaccine Sallu Bhai be like Corona से डर नहीं लगता साहब Radhe के re...,und,"['radhe', 'radhereview']",2597232165,FALSE,9.0,4.0,1.0,"[after, getting, second, dose, of, vaccine, sallu, bhai, be, like, corona, स, डर, नह, लगत, स, हब...","[getting, second, dose, vaccine, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radhe, क, rev...","[get, second, dose, vaccin, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radh, क, review, स..."
2,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje..."
3,14/05/2021 18:08,alokajay chitraaum ravishndtv newstvchannel Dr Ajay Alok ji mera first dose covishield ka laga ...,hi,[],620255405,FALSE,0.0,0.0,0.0,"[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie..."
4,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]"


In [ ]:
import nltk
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

tweet_df['Tweet_lemmatized'] = tweet_df['Tweet_nonstop'].apply(lambda x: lemmatizer(x))
tweet_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]","[going, second, dose, next, sunday]"
1,14/05/2021 20:07,After getting Second Dose of Vaccine Sallu Bhai be like Corona से डर नहीं लगता साहब Radhe के re...,und,"['radhe', 'radhereview']",2597232165,FALSE,9.0,4.0,1.0,"[after, getting, second, dose, of, vaccine, sallu, bhai, be, like, corona, स, डर, नह, लगत, स, हब...","[getting, second, dose, vaccine, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radhe, क, rev...","[get, second, dose, vaccin, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radh, क, review, स...","[getting, second, dose, vaccine, sallu, bhai, like, corona, स, डर, नह, लगत, स, हब, radhe, क, rev..."
2,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj..."
3,14/05/2021 18:08,alokajay chitraaum ravishndtv newstvchannel Dr Ajay Alok ji mera first dose covishield ka laga ...,hi,[],620255405,FALSE,0.0,0.0,0.0,"[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie...","[alokajay, chitraaum, ravishndtv, newstvchannel, dr, ajay, alok, ji, mera, first, dose, covishie..."
4,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]"


In [ ]:
ntweet_df= tweet_df[tweet_df.language == 'en']

In [ ]:
ntweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3176 entries, 0 to 4585
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              3176 non-null   object 
 1   tweet             3176 non-null   object 
 2   language          3176 non-null   object 
 3   hashtags          3176 non-null   object 
 4   user_id           3176 non-null   object 
 5   retweet           3176 non-null   object 
 6   nlikes            3176 non-null   float64
 7   nreplies          3176 non-null   float64
 8   nretweets         3176 non-null   float64
 9   Tweet_tokenized   3176 non-null   object 
 10  Tweet_nonstop     3176 non-null   object 
 11  Tweet_stemmed     3176 non-null   object 
 12  Tweet_lemmatized  3176 non-null   object 
dtypes: float64(3), object(10)
memory usage: 347.4+ KB


In [ ]:
ntweet_df.head()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,14/05/2021 20:25,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]","[going, second, dose, next, sunday]"
1,14/05/2021 18:49,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj..."
2,14/05/2021 14:24,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]"
3,14/05/2021 12:46,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0,"[today, beingsalmankhan, took, his, second, dose, of, vaccine, and, i, took, my, first, dose, of...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...","[today, beingsalmankhan, took, second, dose, vaccin, took, first, dose, vaccin, pinch, httpstcoj...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc..."
4,14/05/2021 10:49,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0,"[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni..."


In [ ]:
ntweet_df.reset_index(drop=True,inplace=True)

In [ ]:
ntweet_df['date'] = ntweet_df['date'].map(lambda x: x[0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ntweet_df

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized
0,14/05/2021,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]","[going, second, dose, next, sunday]"
1,14/05/2021,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj..."
2,14/05/2021,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]"
3,14/05/2021,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0,"[today, beingsalmankhan, took, his, second, dose, of, vaccine, and, i, took, my, first, dose, of...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...","[today, beingsalmankhan, took, second, dose, vaccin, took, first, dose, vaccin, pinch, httpstcoj...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc..."
4,14/05/2021,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0,"[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3171,17/04/2021,ndtv Had an RTPCR done on Wednesday so that I could take a flight for CCU from Bangalore yesterd...,en,[],1528472935,FALSE,0.0,1.0,1.0,"[ndtv, had, an, rtpcr, done, on, wednesday, so, that, i, could, take, a, flight, for, ccu, from,...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalore, yesterday, must, say, proces...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalor, yesterday, must, say, process...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalore, yesterday, must, say, proces..."
3172,16/04/2021,Factche bhupro ndtv AMUofficialPRO UnivofDelhi JNUofficial These are testing times All the insti...,en,[],1.21552E+18,FALSE,1.0,1.0,0.0,"[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, these, are, testing, times, al...","[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, testing, times, institutions, ...","[factch, bhupro, ndtv, amuofficialpro, univofdelhi, jnuoffici, test, time, institut, what, best,...","[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, testing, time, institution, wh..."
3173,16/04/2021,It is scary very scary It has even started shaking faith hope and optimism But thats how testing...,en,"['faith', 'hope', 'optimism', 'thistooshallpass', 'covid19']",1.21552E+18,FALSE,8.0,0.0,0.0,"[it, is, scary, very, scary, it, has, even, started, shaking, faith, hope, and, optimism, but, t...","[scary, scary, even, started, shaking, faith, hope, opti

In [ ]:
size = ntweet_df.shape[0]
size

3176

In [ ]:
from datetime import date

In [ ]:
import datetime
datetime.datetime.strptime(ntweet_df['date'][0], "%d/%m/%Y").strftime("%m%d")

AttributeError: ignored

In [ ]:
ntweet_df['jugaad'] = 0
for i in range(size):
  ntweet_df['jugaad'][i] = datetime.datetime.strptime(ntweet_df['date'][i], "%d/%m/%Y").strftime("%m%d")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

In [ ]:
ntweet_df.head()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized,jugaad
0,14/05/2021,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]","[going, second, dose, next, sunday]",514
1,14/05/2021,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...",514
2,14/05/2021,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]",514
3,14/05/2021,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0,"[today, beingsalmankhan, took, his, second, dose, of, vaccine, and, i, took, my, first, dose, of...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...","[today, beingsalmankhan, took, second, dose, vaccin, took, first, dose, vaccin, pinch, httpstcoj...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...",514
4,14/05/2021,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0,"[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...",514


In [ ]:
ntweet_df.jugaad.describe()

count    3176.00000
mean      492.64767
std        35.65204
min       414.00000
25%       503.00000
50%       511.00000
75%       513.00000
max       514.00000
Name: jugaad, dtype: float64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ntweet_df['dates'] = le.fit_transform(ntweet_df.jugaad)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
ntweet_df.head()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized,jugaad,dates
0,14/05/2021,AN 😼 going for my second dose next Sunday,en,[],3162614958,FALSE,1.0,1.0,0.0,"[an, going, for, my, second, dose, next, sunday]","[going, second, dose, next, sunday]","[go, second, dose, next, sunday]","[going, second, dose, next, sunday]",514,30
1,14/05/2021,AgrawalRMD IndiaToday sardesairajdeep mohanbhagwat DrHVoffice AmitShah CDCgov WHO gavi brajeshli...,en,[],597935746,FALSE,0.0,0.0,0.0,"[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, who, gavi,...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffic, amitshah, cdcgov, gavi, braje...","[agrawalrmd, indiatoday, sardesairajdeep, mohanbhagwat, drhvoffice, amitshah, cdcgov, gavi, braj...",514,30
2,14/05/2021,Still havent got second dose Why this kind of issue httpstcoHgkWOEZy,en,[],951114582,FALSE,0.0,0.0,0.0,"[still, havent, got, second, dose, why, this, kind, of, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]","[still, havent, got, second, dose, kind, issu, httpstcohgkwoezi]","[still, havent, got, second, dose, kind, issue, httpstcohgkwoezy]",514,30
3,14/05/2021,Today BeingSalmanKhan took his second dose of vaccine and I took my first dose of vaccine Same p...,en,[],1.12419E+18,FALSE,9.0,0.0,2.0,"[today, beingsalmankhan, took, his, second, dose, of, vaccine, and, i, took, my, first, dose, of...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...","[today, beingsalmankhan, took, second, dose, vaccin, took, first, dose, vaccin, pinch, httpstcoj...","[today, beingsalmankhan, took, second, dose, vaccine, took, first, dose, vaccine, pinch, httpstc...",514,30
4,14/05/2021,MoHFWINDIA PMOIndia drharshvardhan AshwiniKChoubey PIBIndia mygovindia COVIDNewsByMIB NITIAayog ...,en,[],147840907,FALSE,1.0,0.0,0.0,"[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...","[mohfwindia, pmoindia, drharshvardhan, ashwinikchoubey, pibindia, mygovindia, covidnewsbymib, ni...",514,30


In [ ]:
ntweet_df.tail()

,date,tweet,language,hashtags,user_id,retweet,nlikes,nreplies,nretweets,Tweet_tokenized,Tweet_nonstop,Tweet_stemmed,Tweet_lemmatized,jugaad,dates
3171,17/04/2021,ndtv Had an RTPCR done on Wednesday so that I could take a flight for CCU from Bangalore yesterd...,en,[],1528472935,FALSE,0.0,1.0,1.0,"[ndtv, had, an, rtpcr, done, on, wednesday, so, that, i, could, take, a, flight, for, ccu, from,...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalore, yesterday, must, say, proces...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalor, yesterday, must, say, process...","[ndtv, rtpcr, done, wednesday, could, take, flight, ccu, bangalore, yesterday, must, say, proces...",417,3
3172,16/04/2021,Factche bhupro ndtv AMUofficialPRO UnivofDelhi JNUofficial These are testing times All the insti...,en,[],1.21552E+18,FALSE,1.0,1.0,0.0,"[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, these, are, testing, times, al...","[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, testing, times, institutions, ...","[factch, bhupro, ndtv, amuofficialpro, univofdelhi, jnuoffici, test, time, institut, what, best,...","[factche, bhupro, ndtv, amuofficialpro, univofdelhi, jnuofficial, testing, time, institution, wh...",416,2
3173,16/04/2021,It is scary very scary It has even started shaking faith hope and optimism But thats how testing...,en,"['faith', 'hope', 'optimism', 'thistooshallpass', 'covid19']",1.21552E+18,FALSE,8.0,0.0,0.0,"[it, is, scary, very, scary, it, has, even, started, shaking, faith, hope, and, optimism, but, t...","[scary, scary, even, started, shaking, faith, hope, optimism, thats, testing, times, may, soon, ...","[scari, scari, even, start, shake, faith, hope, optim, that, test, time, may, soon, thistooshall...","[scary, scary, even, started, shaking, faith, hope, optimism, thats, testing, time, may, soon, t...",416,2
3174,15/04/2021,BiharHealthDep NitishKumar i am in jayprabha blood bank and hospital in kankarbagh patna I am h...,en,[],1.3797E+18,FALSE,0.0,0.0,0.0,"[biharhealthdep, nitishkumar, i, am, in, jayprabha, blood, bank, and, hospital, in, kankarbagh, ...","[biharhealthdep, nitishkumar, jayprabha, blood, bank, hospital, kankarbagh, patna, covid, testin...","[biharhealthdep, nitishkumar, jayprabha, blood, bank, hospit, kankarbagh, patna, covid, test, re...","[biharhealthdep, nitishkumar, jayprabha, blood, bank, hospital, kankarbagh, patna, covid, testin...",415,1
3175,14/04/2021,April RahulGandhi said A lockdown is like a pause button it’s not a solutionWhen we come out o...,en,[],3305679984,FALSE,2.0,0.0,0.0,"[, april, rahulgandhi, said, a, lockdown, is, like, a, pause, button, it, s, not, a, solutionwhe...","[, april, rahulgandhi, said, lockdown, like, pause, button, solutionwhen, come, lockdownvirus, s...","[, april, rahulgandhi, said, lockdown, like, paus, button, solutionwhen, come, lockdownviru, sta...","[, april, rahulgandhi, said, lockdown, like, pause, button, solutionwhen, come, lockdownvirus, s...",414,0
